In [8]:
import pretty_midi
import os
import scipy.io.wavfile
import time
import IPython.display as ipd 

In [6]:
# generate the individual instrument wavs for different pitches
note_wavs_dir = "/home/faraaz/workspace/music-transcription/data/note_wavs/"
sf2_path = "/usr/share/sounds/sf2/FluidR3_GM.sf2"

num_programs = 8
min_pitch = 30
max_pitch = 100
note_length = 2
velocity = 80
sample_rate = 44100
start = time.time()

for program in range(num_programs):
    program = program * 8
    for pitch in range(min_pitch, max_pitch+1):
        note = pretty_midi.Note(velocity, pitch, 0, note_length)
        instrument  = pretty_midi.Instrument(program)
        instrument.notes = [note]
        pm = pretty_midi.PrettyMIDI()
        pm.instruments = [instrument]
        
        note_samples = pm.fluidsynth(fs=sample_rate, sf2_path=sf2_path)
        filename = os.path.join(note_wavs_dir, "{}_{}.wav".format(program, pitch))
        scipy.io.wavfile.write(filename, sample_rate, note_samples)
    print("{}. {}s".format(program, time.time()-start))
    start = time.time()
        

0. 7.413976192474365s
8. 7.397974729537964s
16. 7.391552209854126s
24. 7.3399577140808105s
32. 7.330834627151489s
40. 7.419021844863892s
48. 7.430523633956909s
56. 7.3216392993927s


In [9]:
note_wav_file = "/home/faraaz/workspace/music-transcription/data/note_wavs/0_50.wav"
_, note_samples = scipy.io.wavfile.read(note_wav_file)
ipd.Audio(data=note_samples, rate=sample_rate)